In [23]:
import pandas as pd

In [24]:
df = pd.read_csv("mbti_1.csv")

In [25]:
df['posts'] = df['posts'].apply(lambda x: x.split("|||"))

In [26]:
df = df.explode('posts')

In [34]:
df = df.reset_index(drop=True).dropna()
print(df['type'])

0         INFJ
1         INFJ
2         INFJ
3         INFJ
4         INFJ
          ... 
422840    INFP
422841    INFP
422842    INFP
422843    INFP
422844    INFP
Name: type, Length: 422845, dtype: object


In [40]:
df = df[['posts', 'type']]

In [41]:
print(df.iloc[0])

posts    'http://www.youtube.com/watch?v=qsXHcwe3krw
type                                            INFJ
Name: 0, dtype: object


In [42]:

df['input'] = "Given: " + df['posts'] + "MBTI: [MASK]"
df['output'] = "Given: " + df['posts'] + "MBTI: " + df['type']


In [45]:
print(df['output'].iloc[0])

Given: 'http://www.youtube.com/watch?v=qsXHcwe3krwMBTI: INFJ


In [11]:
import pickle

file = open("input_encoding_bert.pkl", 'rb')
data = pickle.load(file)

tensor([[    0, 18377,    35,  ...,     1,     1,     1],
        [    0, 18377,    35,  ...,     1,     1,     1],
        [    0, 18377,    35,  ...,     1,     1,     1],
        ...,
        [    0, 18377,    35,  ...,     1,     1,     1],
        [    0, 18377,    35,  ...,     1,     1,     1],
        [    0, 18377,    35,  ...,     1,     1,     1]])


In [34]:
from datasets import Dataset
import pandas as pd
df = pd.read_csv("converted.csv")
ds = Dataset.from_pandas(df)
from transformers import AutoModelForMultipleChoice, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [35]:
examples = ds[:5]
ending_names = ["ISTJ", "ISFJ", "INFJ", "INTJ", "ISTP", "ISFP", "INFP", "INTP", "ESTP", "ESFP", "ENFP", "ENTP", "ESTJ", "ESFJ", "ENFJ", "ENTJ"]
def preprocess_function(examples):
    first_sentences = [[context] * 16 for context in examples["posts"]]

    question_headers = examples['pad']
    print(question_headers)
    second_sentences = [[f"{header} {end}".strip() for end in ending_names] for i, header in enumerate(question_headers)]
    
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)

    return {k: [v[i:i+16] for i in range(0, len(v), 16)] for k, v in tokenized_examples.items()}


features = preprocess_function(examples)


[' ', ' ', ' ', ' ', ' ']


In [36]:
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 16 [20, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 20]


In [37]:
idx = 2
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(16)]

['<s>On repeat for most of today</s></s>ISTJ</s>',
 '<s>On repeat for most of today</s></s>ISFJ</s>',
 '<s>On repeat for most of today</s></s>INFJ</s>',
 '<s>On repeat for most of today</s></s>INTJ</s>',
 '<s>On repeat for most of today</s></s>ISTP</s>',
 '<s>On repeat for most of today</s></s>ISFP</s>',
 '<s>On repeat for most of today</s></s>INFP</s>',
 '<s>On repeat for most of today</s></s>INTP</s>',
 '<s>On repeat for most of today</s></s>ESTP</s>',
 '<s>On repeat for most of today</s></s>ESFP</s>',
 '<s>On repeat for most of today</s></s>ENFP</s>',
 '<s>On repeat for most of today</s></s>ENTP</s>',
 '<s>On repeat for most of today</s></s>ESTJ</s>',
 '<s>On repeat for most of today</s></s>ESFJ</s>',
 '<s>On repeat for most of today</s></s>ENFJ</s>',
 '<s>On repeat for most of today</s></s>ENTJ</s>']

In [3]:
import pandas as pd
df = pd.read_csv("converted_new.csv")

m = 0
for i in df.iterrows():
    if len(df['posts']) > m:
        m = len(df['posts'])
print(m)

100


In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("roberta-base")
print(model)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [4]:
for i in range(len(model.encoder.layer)):
            if(i < len(model.encoder.layer) + -1):
                for param in model.encoder.layer[i].parameters():
                    if(param.requires_grad):
                        param.requires_grad = False